In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import operator, os
import time, datetime
from utils import *

In [2]:
bath_path = '/mnt/qb/work/mlcolab/hzhou52/kt/assistment/17'
file_name = 'anonymized_full_release_competition_dataset.csv'
log = pd.read_csv(os.path.join(bath_path, file_name), encoding = "utf-8",low_memory=False)

In [3]:
log.columns

Index(['studentId', 'MiddleSchoolId', 'InferredGender', 'SY ASSISTments Usage',
       'AveKnow', 'AveCarelessness', 'AveCorrect', 'NumActions', 'AveResBored',
       'AveResEngcon', 'AveResConf', 'AveResFrust', 'AveResOfftask',
       'AveResGaming', 'action_num', 'skill', 'problemId', 'problemType',
       'assignmentId', 'assistmentId', 'startTime', 'endTime', 'timeTaken',
       'correct', 'original', 'hint', 'hintCount', 'hintTotal', 'scaffold',
       'bottomHint', 'attemptCount', 'frIsHelpRequest', 'frPast5HelpRequest',
       'frPast8HelpRequest', 'stlHintUsed', 'past8BottomOut',
       'totalFrPercentPastWrong', 'totalFrPastWrongCount', 'frPast5WrongCount',
       'frPast8WrongCount', 'totalFrTimeOnSkill', 'timeSinceSkill',
       'frWorkingInSchool', 'totalFrAttempted', 'totalFrSkillOpportunities',
       'responseIsFillIn', 'responseIsChosen', 'endsWithScaffolding',
       'endsWithAutoScaffolding', 'frTimeTakenOnScaffolding',
       'frTotalSkillOpportunitiesScaffolding',
 

In [4]:
log['skill']

0                  properties-of-geometric-figures
1                  properties-of-geometric-figures
2         sum-of-interior-angles-more-than-3-sides
3         sum-of-interior-angles-more-than-3-sides
4         sum-of-interior-angles-more-than-3-sides
                            ...                   
942811                                        mean
942812                                        mean
942813                                        mean
942814                                        mean
942815                        multiplying-decimals
Name: skill, Length: 942816, dtype: object

In [6]:
check_nan(log)
count_unique(log, ['studentId', 'skill', 'problemId'])
print(len(log))

number of NaN in column studentId: 0
number of NaN in column MiddleSchoolId: 0
number of NaN in column InferredGender: 173656
number of NaN in column SY ASSISTments Usage: 0
number of NaN in column AveKnow: 0
number of NaN in column AveCarelessness: 0
number of NaN in column AveCorrect: 0
number of NaN in column NumActions: 0
number of NaN in column AveResBored: 0
number of NaN in column AveResEngcon: 0
number of NaN in column AveResConf: 0
number of NaN in column AveResFrust: 0
number of NaN in column AveResOfftask: 0
number of NaN in column AveResGaming: 0
number of NaN in column action_num: 0
number of NaN in column skill: 0
number of NaN in column problemId: 0
number of NaN in column problemType: 0
number of NaN in column assignmentId: 0
number of NaN in column assistmentId: 0
number of NaN in column startTime: 0
number of NaN in column endTime: 0
number of NaN in column timeTaken: 0
number of NaN in column correct: 0
number of NaN in column original: 0
number of NaN in column hint

In [8]:
# log = log[~log.skill_id.isna()]
log = remove_log(log, 'studentId')
print(len(log))

942816


In [10]:
log['startTime']
log['correct']

0         0
1         1
2         0
3         0
4         1
         ..
942811    1
942812    0
942813    0
942814    1
942815    1
Name: correct, Length: 942816, dtype: int64

# pre-process

In [11]:
# create new data
df = pd.DataFrame(columns=['user_id', 'problem_id', 'skill_id', 'timestamp', 'dwell_time', 'correct'])
df.user_id = log.studentId
df.problem_id = log.problemId
df.skill_id = log.skill
df.timestamp = (log.startTime + log.endTime)/2
df.dwell_time = log.timeTaken
df.correct = log.correct * 1

In [12]:
# re-index
# - skill
skill_id = list(df['skill_id'].unique())
skill_dict = dict(zip(skill_id, range(len(skill_id))))
df['skill_id'] = df['skill_id'].apply(lambda x: skill_dict[x])

# - problem
pro_id = list(df['problem_id'].unique())
pro_dict = dict(zip(pro_id, range(len(pro_id))))
df['problem_id'] = df['problem_id'].apply(lambda x: pro_dict[x])

# - student
stu_id = list(df['user_id'].unique())
stu_dict = dict(zip(stu_id, range(len(stu_id))))
df['user_id'] = df['user_id'].apply(lambda x: stu_dict[x])

In [13]:
df

,user_id,problem_id,skill_id,timestamp,dwell_time,correct
0,0,0,0,1.096470e+09,49.0,0
1,0,1,0,1.096470e+09,4.0,1
2,0,2,1,1.096470e+09,6.0,0
3,0,2,1,1.096470e+09,18.0,0
4,0,3,1,1.096470e+09,2.0,1
...,...,...,...,...,...,...
942811,1708,1777,61,1.147447e+09,4.0,1
942812,1708,1778,61,1.147448e+09,71.0,0
942813,1708,1778,61,1.147448e+09,4.0,0
942814,1708,1778,61,1.147448e+09,3.0,1


In [14]:
df = df.astype({
    'timestamp': np.float64,
    'dwell_time': np.float64,
    'correct': np.float64,
    'problem_id': np.int64,
    'skill_id': np.int64,
    'user_id': np.int64
    })

In [15]:
# Save
df.to_csv(os.path.join(bath_path, 'interactions.csv'), sep='\t', index=False)

In [16]:
count_unique(df, list(df.columns))
len(df)

number of unique value in user_id: 1709
number of unique value in problem_id: 3162
number of unique value in skill_id: 102
number of unique value in timestamp: 807531
number of unique value in dwell_time: 9632
number of unique value in correct: 2


942816